In [1]:
import pandas as pd
import json
import re
from datetime import datetime

# ============================================================================
# STEP 1: Load CSV Data (Orders)
# ============================================================================
print("Step 1: Loading CSV data...")

# Load orders from CSV file
orders_df = pd.read_csv('orders.csv')

# Convert order_date to datetime format for better analysis
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], format='%d-%m-%Y')

print(f"✓ Loaded {len(orders_df)} orders")
print(f"  Columns: {list(orders_df.columns)}")
print(f"\nSample orders data:")
print(orders_df.head(3))


# ============================================================================
# STEP 2: Load JSON Data (Users)
# ============================================================================
print("\n" + "="*70)
print("Step 2: Loading JSON data...")

# Load users from JSON file
with open('users.json', 'r') as f:
    users_data = json.load(f)

# Convert JSON to DataFrame
users_df = pd.DataFrame(users_data)

print(f"✓ Loaded {len(users_df)} users")
print(f"  Columns: {list(users_df.columns)}")
print(f"\nSample users data:")
print(users_df.head(3))


# ============================================================================
# STEP 3: Load SQL Data (Restaurants)
# ============================================================================
print("\n" + "="*70)
print("Step 3: Loading SQL data...")

# Read the SQL file
with open('restaurants.sql', 'r') as f:
    sql_content = f.read()

# Extract INSERT statements using regex
insert_pattern = r"INSERT INTO restaurants VALUES \((\d+), '([^']+)', '([^']+)', ([\d.]+)\);"
matches = re.findall(insert_pattern, sql_content)

# Create DataFrame from extracted data
restaurants_df = pd.DataFrame(matches, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])

# Convert data types
restaurants_df['restaurant_id'] = restaurants_df['restaurant_id'].astype(int)
restaurants_df['rating'] = restaurants_df['rating'].astype(float)

print(f"✓ Loaded {len(restaurants_df)} restaurants")
print(f"  Columns: {list(restaurants_df.columns)}")
print(f"\nSample restaurants data:")
print(restaurants_df.head(3))


# ============================================================================
# STEP 4: Merge the Datasets
# ============================================================================
print("\n" + "="*70)
print("Step 4: Merging datasets...")

# First merge: Orders + Users (LEFT JOIN on user_id)
merged_df = orders_df.merge(
    users_df,
    on='user_id',
    how='left',
    suffixes=('_order', '_user')
)

print(f"✓ After merging orders + users: {len(merged_df)} records")

# Second merge: (Orders + Users) + Restaurants (LEFT JOIN on restaurant_id)
final_df = merged_df.merge(
    restaurants_df,
    on='restaurant_id',
    how='left',
    suffixes=('_order', '_master')
)

print(f"✓ After merging with restaurants: {len(final_df)} records")


# ============================================================================
# STEP 5: Create Final Dataset
# ============================================================================
print("\n" + "="*70)
print("Step 5: Creating final dataset...")

# Select and reorder columns for better readability
final_columns = [
    'order_id',
    'order_date',
    'user_id',
    'name',
    'city',
    'membership',
    'restaurant_id',
    'restaurant_name_order',  # From orders.csv
    'restaurant_name_master',  # From restaurants.sql
    'cuisine',
    'rating',
    'total_amount'
]

final_df = final_df[final_columns]

# Rename columns for clarity
final_df.rename(columns={
    'restaurant_name_order': 'restaurant_name_from_order',
    'restaurant_name_master': 'restaurant_name_from_master'
}, inplace=True)

# Add derived columns for analysis
final_df['order_month'] = final_df['order_date'].dt.month
final_df['order_year'] = final_df['order_date'].dt.year
final_df['order_quarter'] = final_df['order_date'].dt.quarter

print(f"✓ Final dataset prepared with {len(final_df)} records")
print(f"  Total columns: {len(final_df.columns)}")

# Save to CSV
output_file = 'final_food_delivery_dataset.csv'
final_df.to_csv(output_file, index=False)
print(f"\n✓ Saved to: {output_file}")


# ============================================================================
# STEP 6: Data Quality Check
# ============================================================================
print("\n" + "="*70)
print("Data Quality Summary:")
print("="*70)

print("\n1. Missing Values:")
missing_data = final_df.isnull().sum()
print(missing_data[missing_data > 0] if missing_data.sum() > 0 else "  No missing values ✓")

print("\n2. Data Shape:")
print(f"  Rows: {final_df.shape[0]:,}")
print(f"  Columns: {final_df.shape[1]}")

print("\n3. Date Range:")
print(f"  From: {final_df['order_date'].min().strftime('%d-%m-%Y')}")
print(f"  To: {final_df['order_date'].max().strftime('%d-%m-%Y')}")

print("\n4. Value Distributions:")
print(f"  Unique Users: {final_df['user_id'].nunique()}")
print(f"  Unique Restaurants: {final_df['restaurant_id'].nunique()}")
print(f"  Cities: {final_df['city'].nunique()} - {sorted(final_df['city'].unique())}")
print(f"  Cuisines: {final_df['cuisine'].nunique()} - {sorted(final_df['cuisine'].unique())}")

print("\n5. Revenue Summary:")
print(f"  Total Revenue: ₹{final_df['total_amount'].sum():,.2f}")
print(f"  Average Order Value: ₹{final_df['total_amount'].mean():.2f}")
print(f"  Min Order: ₹{final_df['total_amount'].min():.2f}")
print(f"  Max Order: ₹{final_df['total_amount'].max():.2f}")

print("\n6. Membership Distribution:")
print(final_df['membership'].value_counts())


# ============================================================================
# BONUS: Quick Analysis Examples
# ============================================================================
print("\n" + "="*70)
print("Quick Analysis Examples:")
print("="*70)

# Top 5 cities by order count
print("\n1. Top 5 Cities by Order Volume:")
city_orders = final_df['city'].value_counts().head(5)
for city, count in city_orders.items():
    print(f"  {city}: {count:,} orders")

# Top 5 cuisines by revenue
print("\n2. Top 5 Cuisines by Revenue:")
cuisine_revenue = final_df.groupby('cuisine')['total_amount'].sum().sort_values(ascending=False).head(5)
for cuisine, revenue in cuisine_revenue.items():
    print(f"  {cuisine}: ₹{revenue:,.2f}")

# Gold vs Regular membership revenue
print("\n3. Revenue by Membership Type:")
membership_revenue = final_df.groupby('membership')['total_amount'].agg(['sum', 'mean', 'count'])
print(membership_revenue)

# Monthly trend
print("\n4. Orders by Month (2023):")
monthly_orders = final_df[final_df['order_year'] == 2023].groupby('order_month')['order_id'].count()
for month, count in monthly_orders.items():
    month_name = datetime(2023, month, 1).strftime('%B')
    print(f"  {month_name}: {count:,} orders")

print("\n" + "="*70)
print("✓ All done! Dataset is ready for analysis.")
print("="*70)

Step 1: Loading CSV data...
✓ Loaded 10000 orders
  Columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']

Sample orders data:
   order_id  user_id  restaurant_id order_date  total_amount  \
0         1     2508            450 2023-02-18        842.97   
1         2     2693            309 2023-01-18        546.68   
2         3     2084            107 2023-07-15        163.93   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  

Step 2: Loading JSON data...
✓ Loaded 3000 users
  Columns: ['user_id', 'name', 'city', 'membership']

Sample users data:
   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold

Step 3: Loading SQL data...
✓ Loaded 500 restaurants
  Columns: ['restaurant_id', 'restaurant_name', 'cuisine', 'rating']

Sample restauran